In [1]:
LOCAL_MODE = False

# 0. 환경설정

In [2]:
import argparse
import os
import requests
import tempfile
import subprocess, sys

import pandas as pd
import numpy as np
from glob import glob
import copy
from collections import OrderedDict
from pathlib import Path
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

import logging
import logging.handlers

import json
import base64
import boto3
import sagemaker
from botocore.client import Config
from botocore.exceptions import ClientError

import time
from datetime import datetime as dt
import datetime
from pytz import timezone
from dateutil.relativedelta import *

In [3]:
def get_secret():
    secret_name = "dev/ForecastPalmOilPrice"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

keychain = json.loads(get_secret())
ACCESS_KEY_ID = keychain['AWS_ACCESS_KEY_ID']
ACCESS_SECRET_KEY = keychain['AWS_ACCESS_SECRET_KEY']

BUCKET_NAME_USECASE = keychain['PROJECT_BUCKET_NAME']
DATALAKE_BUCKET_NAME = keychain['DATALAKE_BUCKET_NAME']

S3_PATH_REUTER = keychain['S3_PATH_REUTER']
S3_PATH_WWO = keychain['S3_PATH_WWO']
S3_PATH_STAGE = keychain['S3_PATH_STAGE']
S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
S3_PATH_FORECAST = keychain['S3_PATH_PREDICTION']

region = 'ap-northeast-2'
boto3_session = boto3.Session(aws_access_key_id = ACCESS_KEY_ID,
                              aws_secret_access_key = ACCESS_SECRET_KEY,
                              region_name = region)
sm_session = sagemaker.Session(boto_session = boto3_session)

s3_resource = boto3_session.resource('s3')
palmoil_bucket = s3_resource.Bucket(BUCKET_NAME_USECASE)
datalake_bucket = s3_resource.Bucket(DATALAKE_BUCKET_NAME)

sm_client = boto3_session.client('sagemaker')
qs_client = boto3_session.client('quicksight')
s3_client = boto3_session.client('s3')
sts_client = boto3_session.client("sts")

In [4]:
%%writefile src/v1.2/model_validation.py

import glob
import os
import pandas as pd

from collections import defaultdict
import numpy as np
from collections import Counter

import time
from datetime import datetime as dt
import argparse
import json
import boto3
from io import StringIO, BytesIO
import joblib
import sys
import subprocess
import logging
import logging.handlers
import calendar
import tarfile


###############################
######### util 함수 설정 ##########
###############################
def _get_logger():
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  
logger = _get_logger()

def get_secret():
    secret_name = "dev/ForecastPalmOilPrice"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

def check_performance_threshold(iput_df : pd.DataFrame,
                                identifier: str,
                                threshold : float = -100):
    tmp = {}
    satisfied_df = iput_df[iput_df['score_val'] > threshold]
    if len(satisfied_df) > 0:
        tmp['identifier'] = identifier
        tmp['model'] = list(satisfied_df['model'])
        tmp['performance'] = list(satisfied_df['score_val'])
    return tmp

def get_model_performance_report(data):
    result = defaultdict(list)
    models_ext = [row["model"] for row in data if row]
    models = [item for sublist in models_ext for item in sublist]
    performance_ext = [row["performance"] for row in data if row]
    performance = [item for sublist in performance_ext for item in sublist]
    
    count_models = Counter(models)
    
    for keys, values in zip(models, performance):
        result[keys].append(values)

    for key, values in result.items():
        result[key] = []
        result[key].append(count_models[key])
        result[key].append(sum(values) / len(values))
        result[key].append(np.std(values))
    
    # 정렬 1순위 : 비즈니스담당자의 Metric에 선정된 Count 높은 순, 2순위: 표준편차가 작은 순(그래서 -처리해줌)
    result = sorted(result.items(), key=lambda k_v: (k_v[1][0], -k_v[1][2]), reverse=True) 
    return result

def register_model_in_aws_registry(model_zip_path: str,
                                   model_package_group_name: str,
                                   model_description: str,
                                   model_status: str,
                                   sm_client) -> str:
    create_model_package_input_dict = {
        "ModelPackageGroupName": model_package_group_name,
        "ModelPackageDescription": model_description,
        "ModelApprovalStatus": model_status,
        "InferenceSpecification": {
            "Containers": [
                {
                    "Image": '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/autogluon-inference:0.4-cpu-py38',
                    "ModelDataUrl": model_zip_path
                }
            ],
            "SupportedContentTypes": ["text/csv"],
            "SupportedResponseMIMETypes": ["text/csv"],
        }
    }
    create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
    model_package_arn = create_model_package_response["ModelPackageArn"]
    return model_package_arn


def register_manifest(source_path,
                      target_path,
                      s3_client,
                      BUCKET_NAME_USECASE):
    template_json = {"fileLocations": [{"URIPrefixes": []}],
                     "globalUploadSettings": {
                         "format": "CSV",
                         "delimiter": ","
                     }}
    paginator = s3_client.get_paginator('list_objects_v2')
    response_iterator = paginator.paginate(Bucket = BUCKET_NAME_USECASE,
                                           Prefix = source_path.split(BUCKET_NAME_USECASE+'/')[1]
                                          )
    for page in response_iterator:
        for content in page['Contents']:
            template_json['fileLocations'][0]['URIPrefixes'].append(f's3://{BUCKET_NAME_USECASE}/'+content['Key'])
    with open(f'./manifest_testing.manifest', 'w') as f:
        json.dump(template_json, f, indent=2)

    res = s3_client.upload_file('./manifest_testing.manifest',
                                BUCKET_NAME_USECASE,
                                f"{target_path.split(BUCKET_NAME_USECASE+'/')[1]}/visual_validation.manifest")
    return f"{target_path.split(BUCKET_NAME_USECASE+'/')[1]}/visual_validation.manifest"
    
def refresh_of_spice_datasets(user_account_id,
                              qs_data_name,
                              manifest_file_path,
                              BUCKET_NAME_USECASE,
                              qs_client):
    
    ds_list = qs_client.list_data_sources(AwsAccountId='108594546720')
    datasource_ids = [summary["DataSourceId"] for summary in ds_list["DataSources"] if qs_data_name in summary["Name"]]    
    for datasource_id in datasource_ids:
        response = qs_client.update_data_source(
            AwsAccountId=user_account_id,
            DataSourceId=datasource_id,
            Name=qs_data_name,
            DataSourceParameters={
                'S3Parameters': {
                    'ManifestFileLocation': {
                        'Bucket': BUCKET_NAME_USECASE,
                        'Key':  f"{manifest_file_path.split(BUCKET_NAME_USECASE+'/')[1]}/visual_validation.manifest"
                    },
                },
            })
        logger.info(f"datasource_id:{datasource_id} 의 manifest를 업데이트: {response}")
    
    res = qs_client.list_data_sets(AwsAccountId = user_account_id)
    datasets_ids = [summary["DataSetId"] for summary in res["DataSetSummaries"] if qs_data_name in summary["Name"]]
    ingestion_ids = []

    for dataset_id in datasets_ids:
        try:
            ingestion_id = str(calendar.timegm(time.gmtime()))
            qs_client.create_ingestion(DataSetId = dataset_id,
                                       IngestionId = ingestion_id,
                                       AwsAccountId = user_account_id)
            ingestion_ids.append(ingestion_id)
        except Exception as e:
            logger.info(e)
            pass
    for ingestion_id, dataset_id in zip(ingestion_ids, datasets_ids):
        while True:
            response = qs_client.describe_ingestion(DataSetId = dataset_id,
                                                    IngestionId = ingestion_id,
                                                    AwsAccountId = user_account_id)
            if response['Ingestion']['IngestionStatus'] in ('INITIALIZED', 'QUEUED', 'RUNNING'):
                time.sleep(5)     #change sleep time according to your dataset size
            elif response['Ingestion']['IngestionStatus'] == 'COMPLETED':
                print("refresh completed. RowsIngested {0}, RowsDropped {1}, IngestionTimeInSeconds {2}, IngestionSizeInBytes {3}".format(
                    response['Ingestion']['RowInfo']['RowsIngested'],
                    response['Ingestion']['RowInfo']['RowsDropped'],
                    response['Ingestion']['IngestionTimeInSeconds'],
                    response['Ingestion']['IngestionSizeInBytes']))
                break
            else:
                logger.info("refresh failed for {0}! - status {1}".format(dataset_id,
                                                                          response['Ingestion']['IngestionStatus']))
                break
    return response

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--leaderboard_path', type=str, default="/opt/ml/processing/input/leaderboard")   
    parser.add_argument('--model_base_path', type=str)
    parser.add_argument('--manifest_base_path', type=str)
    parser.add_argument('--prediction_base_path', type=str)
    parser.add_argument('--threshold', type=str, default="-100")   
    parser.add_argument('--model_package_group_name', type=str, default = BUCKET_NAME_USECASE)  
    parser.add_argument('--qs_data_name', type=str, default = 'model_result')    

    return parser.parse_args()


if __name__=='__main__':
    logger.info(f"\n### Loading Key value from Secret Manager")
    keychain = json.loads(get_secret())
    ACCESS_KEY_ID = keychain['AWS_ACCESS_KEY_ID']
    ACCESS_SECRET_KEY = keychain['AWS_ACCESS_SECRET_KEY']
    BUCKET_NAME_USECASE = keychain['PROJECT_BUCKET_NAME']
    DATALAKE_BUCKET_NAME = keychain['DATALAKE_BUCKET_NAME']
    S3_PATH_REUTER = keychain['S3_PATH_REUTER']
    S3_PATH_WWO = keychain['S3_PATH_WWO']
    S3_PATH_STAGE = keychain['S3_PATH_STAGE']
    S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
    S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
    S3_PATH_FORECAST = keychain['S3_PATH_PREDICTION']
    
    boto3_session = boto3.Session(aws_access_key_id = ACCESS_KEY_ID,
                                  aws_secret_access_key = ACCESS_SECRET_KEY,
                                  region_name = 'ap-northeast-2')
    
    s3_client = boto3_session.client('s3')
    sm_client = boto3_session.client('sagemaker')
    qs_client = boto3_session.client('quicksight')

    sts_client = boto3_session.client("sts")
    user_account_id = sts_client.get_caller_identity()["Account"]
    ######################################
    ## 커맨드 인자, Hyperparameters 처리 ##
    ######################################
    args = parse_args()
    logger.info("######### Argument Info ####################################")
    logger.info("### start training code")    
    logger.info("### Argument Info ###")
    logger.info(f"args.leaderboard_path: {args.leaderboard_path}")    
    logger.info(f"args.model_base_path: {args.model_base_path}")
    logger.info(f"args.manifest_base_path: {args.manifest_base_path}")
    logger.info(f"args.prediction_base_path: {args.prediction_base_path}")
    logger.info(f"args.threshold: {args.threshold}")
    logger.info(f"args.model_package_group_name: {args.model_package_group_name}")
    logger.info(f"args.qs_data_name: {args.qs_data_name}")
  
    leaderboard_path = args.leaderboard_path
    model_base_path = args.model_base_path
    manifest_base_path = args.manifest_base_path
    prediction_base_path = args.prediction_base_path
    threshold = float(args.threshold)
    model_package_group_name = args.model_package_group_name
    qs_data_name = args.qs_data_name
    
    lb_list = sorted(os.listdir(leaderboard_path))
    logger.info(f"leaderboard file list in {leaderboard_path}: {lb_list}")
    satisfied_info = []
    
    for idx, f_path in enumerate(lb_list):
        leaderboard = pd.read_csv(f'{leaderboard_path}/{f_path}').sort_values(by = ['score_val', 'score_test'],
                                                                              ascending = False)
        satisfied_info.append(check_performance_threshold(iput_df = leaderboard,
                                                          identifier = f'fold{idx}',
                                                          threshold = threshold))
    model_report = get_model_performance_report(satisfied_info)

    if model_report[0][1][0] == len(lb_list): # Fold 내 모든 성능이 비즈니스 담당자가 설정한 값을 만족한다면
        logger.info(f"\n#### Pass the 1st minimum performance valiation")
        manifest_file_path = register_manifest(prediction_base_path, 
                                               manifest_base_path,
                                               s3_client,
                                     BUCKET_NAME_USECASE)
        model_package_arn = register_model_in_aws_registry(f"{model_base_path}/model.tar.gz",
                                                           model_package_group_name,
                                                           ','.join(map(str,  model_report[0])),
                                                           'PendingManualApproval',
                                                           sm_client)
        logger.info('### Passed ModelPackage Version ARN : {}'.format(model_package_arn))
        res = refresh_of_spice_datasets(user_account_id,
                                        qs_data_name,
                                        manifest_file_path,
                                        BUCKET_NAME_USECASE,
                                        qs_client)
        logger.info('### refresh_of_spice_datasets : {}'.format(res))
    else:
        logger.info(f"\n#### Filtered at 1st valiation")
        model_package_arn = register_model_in_aws_registry(f"{model_base_path}/model.tar.gz",
                                                           model_package_group_name,
                                                           ','.join(map(str,  model_report[0])),
                                                           'Rejected',
                                                           sm_client)
        logger.info('### Rejected ModelPackage Version ARN : {}'.format(model_package_arn))

Overwriting src/v1.2/model_validation.py


In [5]:
!aws s3 cp 'src/v1.2/model_validation.py' 's3://crude-palm-oil-prices-forecast/src/model_validation.py' --exclude ".ipynb_checkpoints*"

upload: src/v1.2/model_validation.py to s3://crude-palm-oil-prices-forecast/src/model_validation.py


In [6]:
model_validation_code = 's3://crude-palm-oil-prices-forecast/src/model_validation.py'
%store model_validation_code

Stored 'model_validation_code' (str)


In [7]:
%store

Stored variables and their in-db values:
model_validation_code             -> 's3://crude-palm-oil-prices-forecast/src/model_val


In [8]:
%store -r

# 1. 모델 검증 파이프라인 의 스텝(Step) 생성
## 1) 모델 검증 파이프라인 변수 생성
파이프라인에서 사용할 파이프라인 파라미터를 정의합니다. 파이프라인을 스케줄하고 실행할 때 파라미터를 이용하여 실행조건을 커스마이징할 수 있습니다. 파라미터를 이용하면 파이프라인 실행시마다 매번 파이프라인 정의를 수정하지 않아도 됩니다.

지원되는 파라미터 타입은 다음과 같습니다:

- ParameterString - 파이썬 타입에서 str
- ParameterInteger - 파이썬 타입에서 int
- ParameterFloat - 파이썬 타입에서 float
이들 파라미터를 정의할 때 디폴트 값을 지정할 수 있으며 파이프라인 실행시 재지정할 수도 있습니다. 지정하는 디폴트 값은 파라미터 타입과 일치하여야 합니다.

본 노트북에서 사용하는 파라미터는 다음과 같습니다.

- processing_instance_type - 프로세싱 작업에서 사용할 ml.* 인스턴스 타입
- processing_instance_count - 프로세싱 작업에서 사용할 인스턴스 개수
- validation_instance_type - 학습작업에서 사용할 ml.* 인스턴스 타입
- model_approval_status - 학습된 모델을 CI/CD를 목적으로 등록할 때의 승인 상태 (디폴트는 "PendingManualApproval")
- input_data - 입력데이터에 대한 S3 버킷 URI
파이프라인의 각 스텝에서 사용할 변수를 파라미터 변수로서 정의 합니다.

In [10]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
model_validation_instance_count = ParameterInteger(
    name="ModelValidationInstanceCount",
    default_value=1
)
model_validation_instance_type = ParameterString(
    name="ModelValidationInstanceType",
    default_value='ml.c5.xlarge'
)
input_leaderboard_data = ParameterString(
    name="InputLeaderboardPath",
    default_value = 's3://crude-palm-oil-prices-forecast/trained-model/2023/02/26/1677484312.0/leaderboard',
)

## 2) 로컬에서 테스트

In [11]:
if LOCAL_MODE:
    # 도커 컨테이너 입력 폴더: staged data가 들어가는 부분
    base_input_dir = 'opt/ml/processing/input'
    os.makedirs(base_input_dir, exist_ok=True)
    
    # 도커 컨테이너 모델 폴더: model 데이터가 압축해제되고 실행되는곳
    base_model_dir = 'opt/ml/model' 
    os.makedirs(base_model_dir, exist_ok=True)

    # 도커 컨테이너 모델 폴더: prediction 데이터가 압축해제되고 실행되는곳
    base_input_prediction_dir = 'opt/ml/processing/prediction'
    os.makedirs(base_input_prediction_dir, exist_ok=True)
        
    # 도커 컨테이너 기본 출력 폴더
    base_output_dir = 'opt/ml/processing/output'
    os.makedirs(base_output_dir, exist_ok=True)

    # 도커 컨테이너 출력 폴더: stage 데이터셋이 들어가는 부분
    base_output_manifest_dir = f'{base_output_dir}/manifest'
    os.makedirs(base_output_manifest_dir, exist_ok=True)

In [12]:
# !python src/v1.2/model_validation.py

## 3) 모델 검증 프로세서 정의
전처리의 내장 SKLearnProcessor 를 통해서 sklearn_processor 오브젝트를 생성 합니다.

In [13]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version = framework_version,
    instance_type = model_validation_instance_type,
    instance_count = model_validation_instance_count,
    base_job_name = f"{BUCKET_NAME_USECASE}(Validation Model)",
    role = sagemaker.get_execution_role(),
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


## 4) 모델 검증 단계 정의
처리 단계에서는 아래와 같은 주요 인자가 있습니다.
단계 이름
- processor 기술: 위에서 생성한 processor 오브젝트를 제공
- inputs: S3의 경로를 기술하고, 다커안에서의 다운로드 폴더(destination)을 기술 합니다.
- outputs: 처리 결과가 저장될 다커안에서의 폴더 경로를 기술합니다.

도커안의 결과 파일이 저장 후에 자동으로 S3로 업로딩을 합니다.
- job_arguments: 사용자 정의의 인자를 기술 합니다.
- code: 전처리 코드의 경로를 기술 합니다.
처리 단계의 상세한 사항은 여기를 보세요. --> 처리 단계, Processing Step

In [14]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

step_model_validaion = ProcessingStep(
    name = f"{BUCKET_NAME_USECASE}-Validation",
    processor = sklearn_processor,
    inputs=[
            ProcessingInput(
                source = input_leaderboard_data,
                destination = "/opt/ml/processing/input/leaderboard"),
        ],
    job_arguments=["--model_base_path", model_base_path,
                   "--manifest_base_path", manifest_base_path,
                   "--prediction_base_path", 's3://crude-palm-oil-prices-forecast/trained-model/2023/02/26/1677484312.0/prediction',
                   "--threshold", "-100",
                   "--model_package_group_name", BUCKET_NAME_USECASE,
                   "--qs_data_name", "model_result",
                  ],
    code = model_validation_code
)

NameError: name 'model_base_path' is not defined

## 5) 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행
이제 지금까지 생성한 단계들을 하나의 파이프라인으로 조합하고 실행하도록 하겠습니다.

파이프라인은 name, parameters, steps 속성이 필수적으로 필요합니다. 여기서 파이프라인의 이름은 (account, region) 조합에 대하여 유일(unique))해야 합니다.

주의:

- 정의에 사용한 모든 파라미터가 존재해야 합니다.
- 파이프라인으로 전달된 단계(step)들은 실행순서와는 무관합니다. SageMaker Pipeline은 단계가 실행되고 완료될 수 있도록 의존관계를를 해석합니다.
- [알림] 정의한 stpes 이 복수개이면 복수개를 기술합니다. 만약에 step 간에 의존성이 있으면, 명시적으로 기술하지 않아도 같이 실행 됩니다.

### 5-1) 파이프라인 정의

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(name = pipeline_name,
                    parameters = [
                        model_validation_instance_type, 
                        model_validation_instance_count,
                        input_leaderboard_data,
                    ],
                    steps = [step_model_validaion],
)

### 5-2) 파이프라인 정의 확인

In [ ]:
import json

definition = json.loads(pipeline.definition())
definition

### 5-3) 파이프라인 정의를 제출하고 실행하기
파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.

In [ ]:
%%time
start = time.time()
pipeline.upsert(role_arn=sagemaker.get_execution_role())
execution = pipeline.start()
execution.wait() #실행이 완료될 때까지 기다린다.
end = time.time()

In [ ]:
print(f"model validation 시간시간 : {((end - start)/60):.1f} min({end - start:.1f} sec)")

- 2022년 11월 26일 Model validation : 4.5min
- 2023년 03월 01일 Model validation : 4.6 min(273.5 sec)

In [ ]:
execution.describe()

In [ ]:
#실행된 단계들을 리스트업. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여준다.
execution.list_steps()

In [ ]:
response = execution.list_steps()
proc_arn = response[-1]['Metadata']['ProcessingJob']['Arn'] # index -1은 가장 처음 실행 step
proc_job_name = proc_arn.split('/')[-1] # Processing job name만 추출
response = sm_client.describe_processing_job(ProcessingJobName = proc_job_name)
response